# Aprenda a processar um documento XML
Muitos dos documentos que encontramos com informações hoje em dia são XML.

Processá-los exige algum documento específico que você terá que construir para participar nesse curso.

Para provar que aprendeu XML você deve fazer uma pequena atividade.

1. Baixe a nossa Base Oficial de Trabalho (CysticFibrosis2)
2. Escolha dois processadores de XML diferentes em Python, A e B (DOM, SAX ou outro)
3. Use um processador A para obter o nome de todos os AUTHOR citados no arquivo cf79.xml, salve 1 por linha no arquivo autores.xml
4. Use um processador B para obter o nome de todos os TITLE citados no arquivo cf79.xml, salve 1 por linha no arquivo titulo.xml

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

#Considering that data file is in your Google Drive
xml_file_path = '/content/gdrive/My Drive/datasets/CysticFibrosis2-20220410/cf79.xml'

Mounted at /content/gdrive


# Pegar AUTHOR usando ElementTree DOM

Manipulações com DOM são práticas e mais simples. No entanto, não funcionam bem se o documento XML for muito grande, porque o DOM é carregado na memória RAM.

In [11]:
import xml.etree.ElementTree as ET

tree = ET.parse(xml_file_path)
root = tree.getroot()

In [12]:
authors_root = ET.Element("AUTHORS")

for author in root.iter('AUTHOR'):
  ET.SubElement(authors_root, "AUTHOR").text = author.text

authors_tree = ET.ElementTree(authors_root)
authors_tree.write("autores.xml", xml_declaration=True)

# Pegar TITLE com SAX

Parsing utilizando SAX são mais verbosos que com DOM. No entanto, são mais recomendados para grandes arquivos XML, uma vez que não salvam a árvore inteira na memória, tratando o arquivo de forma iterativa em streams de dados.

In [18]:
from xml.sax import parse
from xml.sax.handler import ContentHandler

class TitleHandler(ContentHandler):
    def __init__(self, titles_root):
        super().__init__()
        self.titles_root = titles_root
        self.current_tag = ""
        self.current_content = ""
    
    def startElement(self, tag, attrs):
      self.current_tag = tag

    def endElement(self, tag):
      if self.current_tag == "TITLE":
        ET.SubElement(self.titles_root, "TITLE").text = self.current_content
      self.current_tag = ""
      self.current_content = ""

    def characters(self, content):
      self.current_content += content

titles_root = ET.Element("TITLES")
parse(xml_file_path, TitleHandler(titles_root))

titles_tree = ET.ElementTree(titles_root)
titles_tree.write("titulo.xml", xml_declaration=True)